# AUTO GENERATION OF SUBJECTIVE ANSWERS

### Step 1:  Raw Text Data to Organized dataframe
### Step 2:  Paragraph-wise Keyword Extraction
### Step 3:  Vectorizing Keywords to form Representative Vectors for paragraphs
### Step 4:  Summarizing Paragraphs to generate fixed length Answers
### Step 5:  Query Question to Vector
### Step 6:  Cosine Similarity to find required paragraph

# 

In [1]:
import re
import spacy
import yake
import nltk
import string
import gensim
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from rake_nltk import Rake
from keybert import KeyBERT
from termcolor import colored
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from __future__ import absolute_import
from nltk.tokenize import sent_tokenize
from sumy.parsers.html import HtmlParser
from gensim.summarization import keywords
from sumy.nlp.tokenizers import Tokenizer
from gensim.summarization import keywords
from gensim.summarization import summarize
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sklearn.metrics.pairwise import cosine_similarity
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from __future__ import division, print_function, unicode_literals

## <font color='blue'>Step1: Raw Text Data to Organized dataframe</font>

In [2]:
para_data = [[]]
para_name = ''
chapter_name = ''
EndOfChap = 0
file = "Our_Past_1_Chapter2.txt"
with open(file, encoding="utf8") as f:
    for count, line in enumerate(f):
        if EndOfChap < 2:
            if 'CHAPTER' in line:
                chapter_name = line.split(':')[1].strip()
                continue
                
            if line == '\n':
                para_data.append([])
                EndOfChap +=1
                
            elif para_data[-1] == [] and para_name == '':
                para_name = line
                
            else:
                para_descr = line
                para_data[-1] = [chapter_name, para_name, para_descr]
                para_name = ''
                EndOfChap  = 0
                

para_data = [i for i in para_data if i != []]
for o_index in range(len(para_data)):
    for i_index in range(len(para_data[o_index])):
        para_data[o_index][i_index] = para_data[o_index][i_index].strip()

In [3]:
a = []
count = 0
sent_data = []
para_name = ''
chapter_name = ''
EndOfChap = 0
newpara = False

nlp = spacy.load("en_core_web_sm")
with open(file, encoding="utf8") as f:
    for line in f:
        a.append(line)

for index,line in enumerate(a):
    if EndOfChap < 2:
        if 'CHAPTER' in line:
            chapter_name = line.split(':')[1].strip()
            continue
            
        if line == '\n':
            para_name = ''
            newpara = True
            EndOfChap +=1
            continue
        
        if newpara and para_name == '':
            para_name = line.strip()
            newpara = False
            continue
            
        if not newpara:
            doc = nlp(a[index])
            for sent in doc.sents:
                fsent = str(sent).strip()
                sent_data.append([chapter_name,para_name,fsent])
            EndOfChap  = 0

In [4]:
df_sent = pd.DataFrame(sent_data, columns = ['Chapter Name', 'Paragraph Title', 'Sentence'])
df_para = pd.DataFrame(para_data, columns = ['Chapter Name', 'Paragraph Title', 'Description'])
df_para['Description'] = df_para['Description'].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
df_sent.index = np.arange(1,len(df_sent)+1)
df_para.index = np.arange(1,len(df_para)+1)

## <font color='blue'>Step 2: Paragraph-wise Keyword Extraction</font>

In [5]:
def spacy_kw_ext(text):
    nlp = spacy.load("en_core_web_lg")
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)

    final_kw =  [(x[0]) for x in Counter(result).most_common(10)]
    return final_kw

In [6]:
def yake_kw_ext(text):
    kw_extractor = yake.KeywordExtractor()
    final_kw = []
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 10
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    for kw in keywords[::-1]:
        final_kw.append(kw[0])
    return final_kw

In [7]:
def rakenltk_kw_ext(text):
    rake_nltk_var = Rake(min_length=1, max_length=1)
    rake_nltk_var.extract_keywords_from_text(text)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()
    return keyword_extracted[:10]

In [8]:
def gensim_kw_ext(text):
    k = keywords(text, words = 10, lemmatize = True,).split('\n')
    final_kw = []
    for i in k:
        final_kw+=i.split(' ')
    return final_kw

In [9]:
def KeyBert_distilbert(text):
    kw = []
    kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = kw_model.extract_keywords(text)
    temp = kw_model.extract_keywords(text, keyphrase_ngram_range=(1,1), stop_words='english', use_maxsum=True, top_n=10, diversity=0.2, nr_candidates = 20)
    for i in temp:
        kw.append(i[0])
    return kw

In [10]:
def KeyBert_paraphrase(text):
    kw = []
    kw_model = KeyBERT(model='xlm-r-distilroberta-base-paraphrase-v1')
    keywords = kw_model.extract_keywords(text)
    temp = kw_model.extract_keywords(text, keyphrase_ngram_range=(1,1), stop_words='english', use_maxsum=True, diversity=0.7, top_n = 10)
    for i in temp:
        kw.append(i[0])
    return kw

In [11]:
df_keywords = pd.DataFrame()
df_keywords['Spacy'] = df_para['Description'].apply(spacy_kw_ext)
df_keywords['Yake'] = df_para['Description'].apply(yake_kw_ext)
df_keywords['RakeNLTK'] = df_para['Description'].apply(rakenltk_kw_ext)
df_keywords['Gensim'] = df_para['Description'].apply(gensim_kw_ext)
df_keywords['Keybert-distilbert'] = df_para['Description'].apply(KeyBert_distilbert)
df_keywords['Keybert-paraphrase'] = df_para['Description'].apply(KeyBert_paraphrase)

## <font color='blue'>Step 3: Vectorizing Keywords to form Representative Vectors for paragraps</font>

In [12]:
with open(file,encoding = 'utf8') as f:  
    text = f.read()
    processed_text = text.lower()
    processed_text = processed_text.replace("-", "")
    processed_text = re.sub('[^a-zA-Z]', ' ', processed_text)
    processed_text = re.sub(r'\s+', ' ', processed_text)

all_sentences = nltk.sent_tokenize(processed_text)
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [13]:
model1 = Word2Vec(all_words, size=100, window=20, min_count=1, workers=4)
model2 = Word2Vec(all_words, min_count=1)

In [14]:
def Vectorize(kwlist, model):
    ans = np.zeros(shape=(100,))
    count = 0
    for i in kwlist:
        i = re.sub('[^a-zA-Z]', '',i)
        try:
            ans += model.wv[i.lower()]
            count +=1
        except:
            continue
    return ans/count

In [15]:
df_vectorized = pd.DataFrame()

df_vectorized['Spacy_Vectorized-M1'] = df_keywords['Spacy'].apply(Vectorize, args = (model1,))
df_vectorized['Yake_Vectorized-M1'] = df_keywords['Yake'].apply(Vectorize, args = (model1,))
df_vectorized['RakeNLTK_Vectorized-M1'] = df_keywords['RakeNLTK'].apply(Vectorize, args = (model1,))
df_vectorized['Gensim_Vectorized-M1'] = df_keywords['Gensim'].apply(Vectorize, args = (model1,))
df_vectorized['Keybert-distilbert_Vectorized-M1'] = df_keywords['Keybert-distilbert'].apply(Vectorize, args = (model1,))
df_vectorized['Keybert-paraphrase_Vectorized-M1'] = df_keywords['Keybert-paraphrase'].apply(Vectorize, args = (model1,))

df_vectorized['Spacy_Vectorized-M2'] = df_keywords['Spacy'].apply(Vectorize, args = (model2,))
df_vectorized['Yake_Vectorized-M2'] = df_keywords['Yake'].apply(Vectorize, args = (model2,))
df_vectorized['RakeNLTK_Vectorized-M2'] = df_keywords['RakeNLTK'].apply(Vectorize, args = (model2,))
df_vectorized['Gensim_Vectorized-M2'] = df_keywords['Gensim'].apply(Vectorize, args = (model2,))
df_vectorized['Keybert-distilbert_Vectorized-M2'] = df_keywords['Keybert-distilbert'].apply(Vectorize, args = (model2,))
df_vectorized['Keybert-paraphrase_Vectorized-M2'] = df_keywords['Keybert-paraphrase'].apply(Vectorize, args = (model2,))

## <font color='blue'> Step 4:  Summarizing Paragraphs to generate fixed length Answers</font>

In [16]:
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def Custom_Summary(sentences,n):
    word_embeddings = {}
    sentence_vectors = []
    summary = ''
    
    f1 = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f1:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    
    clean_sentences = pd.Series(sentences).str.strip().replace("[^a-zA-Z]", " ")   
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    for i in range(n):
        summary = summary + ranked_sentences[i][1] + ' '

    return summary.strip()

In [17]:
def Gensim_Summary(sentences,wc):
    contents = ' '.join(sentences)
    return summarize(contents, word_count= wc)

In [18]:
def LexRank_Summary(sentences,n):
    LANG = "english"
    ans = ''
    contents = ' '.join(sentences)
    parser = PlaintextParser.from_string(contents, Tokenizer(LANG))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, n) 
    for sentence in summary:
        ans = ans + str(sentence) + ' '
    return ans

In [19]:
def Luhn_Summary(sentences,n):
    LANG = "english"
    ans = ''
    contents = ' '.join(sentences)
    parser = PlaintextParser.from_string(contents, Tokenizer(LANG))
    summarizer = LuhnSummarizer()
    summary = summarizer(parser.document, n) 
    for sentence in summary:
        ans = ans + str(sentence) + ' '
    return ans

In [20]:
def LSA_Summary(sentences,n):
    LANG = "english"
    ans = ''
    contents = ' '.join(sentences)
    parser = PlaintextParser.from_string(contents, Tokenizer(LANG))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, n) 
    for sentence in summary:
        ans = ans + str(sentence) + ' '
    return ans

In [21]:
def make_sent(text_data):
    sents = nltk.sent_tokenize(text_data)
    return sents

In [22]:
df_sent = pd.DataFrame(sent_data, columns = ['Chapter Name', 'Paragraph Title', 'Sentence'])
df_para = pd.DataFrame(para_data, columns = ['Chapter Name', 'Paragraph Title', 'Description'])
df_sent.index = np.arange(1,len(df_sent)+1)
df_para.index = np.arange(1,len(df_para)+1)
df_summarized = pd.DataFrame()
df_summarized['Sentence List'] = df_para['Description'].apply(make_sent)

In [23]:
df_summarized['Custom Summarizer'] = df_summarized['Sentence List'].apply(Custom_Summary, args = (3,))
df_summarized['Gensim Summarizer'] = df_summarized['Sentence List'].apply(Gensim_Summary, args = (50,))
df_summarized['LexRank Summarizer'] = df_summarized['Sentence List'].apply(LexRank_Summary, args = (3,))
df_summarized['Luhn Summarizer'] = df_summarized['Sentence List'].apply(Luhn_Summary, args = (3,))
df_summarized['LSA Summarizer'] = df_summarized['Sentence List'].apply(LSA_Summary, args = (3,))

## <font color='blue'> Step 5: Query Question to Vector</font>

In [24]:
def Vectorize_Question(question, method = 'Spacy', model = model2):
    if (method == 'Yake'):
        question_kw = yake_kw_ext(question)
    elif (method == 'Rakenltk'):
        question_kw = rakenltk_kw_ext(question)
    elif (method == 'Gensim'):
        question_kw = gensim_kw_ext(question)
    elif (method == 'Kb-distil'):
        question_kw = KeyBert_distilbert(question)
    elif (method == 'Kb-para'):
        question_kw = KeyBert_paraphrase(question)
    else:
        question_kw = spacy_kw_ext(question)
    #print(question_kw)
    question_vec = Vectorize(question_kw, model)    
    return question_vec

## <font color='blue'>Step 6: Cosine Similiarity to find required paragraph</font>

In [25]:
def Para_Vec_Scorer( question_vec, model, method = 'Spacy'):
    para_id = 0
    para_score = []
    
    if (method == 'Yake'):
        mvec = 'Yake_Vectorized'
    elif (method == 'RakeNLTK'):
        mvec = 'RakeNLTK_Vectorized'
    elif (method == 'Gensim'):
        mvec = 'Gensim_Vectorized'
    elif (method == 'Keybert-distilbert'):
        mvec = 'Keybert-distilbert_Vectorized'
    elif (method == 'Keybert-paraphrase'):
        mvec = 'Keybert-paraphrase_Vectorized'
    else:
        mvec = 'Spacy_Vectorized'
        
    if (model == 'Model1'):
        mvec = mvec + '-' + 'M1'
    else:
        mvec = mvec + '-' +'M2'   
    for idx,vec in enumerate(df_vectorized[mvec]):
        curr = np.dot(question_vec, vec) / (np.linalg.norm(question_vec) * np.linalg.norm(vec))
        para_score.append(curr)
    return para_score

In [26]:
def Para_Title_Scorer(q):
    title_tokens = []
    q_tokens = []
    para_title_score = [0]
    sw_list = ['where', 'what', 'why', 'which', 'how', 'describe', 'details', 'detailed', 'list', 'give']

    all_stopwords = stopwords.words('english')
    all_stopwords.extend(sw_list)

    for i in df_para['Paragraph Title']:
        i = i.translate(str.maketrans('', '', string.punctuation))
        doc = nlp(i)
        title_tokens.append([token.lemma_.lower() for token in doc])

    q = q.translate(str.maketrans('', '', string.punctuation))
    doc = nlp(q)
    for token in doc:
        if token.lemma_ not in all_stopwords:
            q_tokens.append(token.lemma_.lower())
    q_tokens

    for i in title_tokens:
        for j in q_tokens:
            if j in i:
                para_title_score[-1]+=1
        para_title_score[-1] /= len(q_tokens)
        para_title_score.append(0)
        
    return para_title_score

In [27]:
def Find_Para(para_score, para_title_score):
    final_score = []
    for i in range(len(para_score)):
        final_score.append(para_score[i] + para_title_score[i])
    
    return final_score.index(max(final_score)) + 1

In [28]:
def Get_Answer(para_id,summary_method):
    temp = summary_method + ' Summarizer'
    if para_id == 0:
        return 'No Valid Answer found'
    return df_summarized[temp][para_id]

## <font color='Dark orange'>Complete Implementation</font>

In [31]:
kw_dict = {1: "Spacy", 2: "Yake", 3: "RakeNLTK", 4: "Gensim", 5: "Keybert-distilbert", 6: "Keybert-paraphrase"}
sum_dict = {1: "Custom", 2: "Gensim", 3: "LexRank", 4: "Luhn", 5: "LSA"}
model_dict = {1: "model1", 2: "model2"}
model_dict_2 = {1: model1, 2: model2}


print(colored('SUBJECTIVE ANSWER FINDER','blue'))
Question = input('Please type your Query Question: ')

print('\nPlease select a keyword Extraction method:')
while True:
    print(colored('1.Spacy\t\t 2.Yake\t\t 3.RakeNLTK\t\t 4.Gensim\t\t 4.KB-distilbert\t\t 5.KB-paraphrase','green'))
    try:
        Kw_Extraction_Method = int(input())
    except ValueError:
        print(colored("Please enter Valid Choice", 'red'))
        continue
    if Kw_Extraction_Method not in [1,2,3,4,5]:
        print(colored('Please Choose a valid Keyword Extraction Method', 'red'))
        continue
    break
print('\nPlease select a Summarization method:')
while True:
    print(colored('1.Custom\t\t 2.Gensim\t\t 3.LexRank\t\t 4.Luhn\t\t 5.LSA', 'green'))
    try:
        Summarization_Method = int(input())
    except ValueError:
        print(colored("Please enter Valid Choice", 'red'))
        continue
    if Summarization_Method not in [1,2,3,4,5]:
        print(colored('Please Choose a valid Summarizer', 'red'))
        continue
    break
print('\nPlease select a Model')
while True:
    print(colored('1.M1 --> More Selective\t\t 2.M2 --> Less Selective', 'green'))
    try:
        Chosen_Model = int(input())
    except ValueError:
        print(colored("Please enter Valid Choice", 'red'))
        continue
    if Chosen_Model not in [1,2]:
        print(colored('Please Choose correct Model', 'red'))
        continue
    break
print(colored('\n\nYour choices are:', 'blue'))
print("Question: ", Question)
print("Keyword Extractor: ", kw_dict[Kw_Extraction_Method])
print("Summarizer: ", sum_dict[Summarization_Method])
print("Selected Model: ", model_dict[Chosen_Model])

while True:
    try:
        choice1 = input('\nDo you wish to proceed (y or n): ')
    except ValueError:
        print(colored("Please enter Valid Choice", 'red'))
        continue
    choice1 = choice1.lower()
    if choice1 not in ['y','n']:
        print(colored('Please choose a Valid Choice', 'red'))
        continue
    break
    
if (choice1 == 'n'):
    print(colored('\nRun Again to Change Choices', 'red'))
else:
    qvec = Vectorize_Question(question = Question, model = model_dict_2[Chosen_Model], method = kw_dict[Kw_Extraction_Method])
    p_sc = Para_Vec_Scorer(qvec, model_dict[Chosen_Model], method = kw_dict[Kw_Extraction_Method])
    pt_sc = Para_Title_Scorer(Question)
    pid = Find_Para(p_sc, pt_sc)
    answer = Get_Answer(pid, sum_dict[Summarization_Method])
    
    print("-------------------------------------------------------------------------------------------------------------")
    print(colored('Complete Answer', 'blue'))
    print("-------------------------------------------------------------------------------------------------------------")
    print(df_para['Paragraph Title'][pid])
    print(df_para['Description'][pid])
    
    print("--------------------------------------------------------------------------------------------------------------")    
    print(colored('Summarized Answer:', 'blue'))
    print("-------------------------------------------------------------------------------------------------------------")
    print(df_para['Paragraph Title'][pid])
    print(answer)
        

SUBJECTIVE ANSWER FINDER
Please type your Query Question: How was stone wood and bones used by hunter gatherers ?

Please select a keyword Extraction method:
1.Spacy		 2.Yake		 3.RakeNLTK		 4.Gensim		 4.KB-distilbert		 5.KB-paraphrase
1

Please select a Summarization method:
1.Custom		 2.Gensim		 3.LexRank		 4.Luhn		 5.LSA
2

Please select a Model
1.M1 --> More Selective		 2.M2 --> Less Selective
2


Your choices are:
Question:  How was stone wood and bones used by hunter gatherers ?
Keyword Extractor:  Spacy
Summarizer:  Gensim
Selected Model:  model2

Do you wish to proceed (y or n): y
-------------------------------------------------------------------------------------------------------------
Complete Answer
-------------------------------------------------------------------------------------------------------------
How do we know about this place ?
Archaeologists have found some of the things hunter gatherers made and used. It is likely that people made and used tools of stone, woo

In [ ]:
kw_dict = {1: "Spacy", 2: "Yake", 3: "RakeNLTK", 4: "Gensim", 5: "Keybert-distilbert", 6: "Keybert-paraphrase"}
sum_dict = {1: "Custom", 2: "Gensim", 3: "LexRank", 4: "Luhn", 5: "LSA"}
model_dict = {1: "model1", 2: "model2"}
model_dict_2 = {1: model1, 2: model2}

In [30]:
## Testing The accuracy:

#kw_dict = {1: "Spacy", 2: "Yake"}
kw_dict = {1: "Spacy", 2: "Yake", 3: "RakeNLTK", 4: "Gensim", 5: "Keybert-distilbert", 6: "Keybert-paraphrase"}
sum_dict = {1: "Custom", 2: "Gensim", 3: "LexRank", 4: "Luhn", 5: "LSA"}
model_dict = {1: "model2"}
model_dict_2 = {1: model2}

q1 = "Give details of Tushar's journey"
q2 = 'Describe the life of Hunter Gatherers.'
q3 = 'How was stone wood and bones used by hunter gatherers ?'
q4 = 'Give some information about Rock paintings'
q5 = 'What are sites ?'
q6 = 'Give uses of fire'
q7 = 'What is Palaeolithic period ?'
q8 = 'What were the changes in climate and environment ?'
q9 = 'Discuss Making Stone Tools by the people of Stone Age.'
q10 = 'Discuss in short the art of Rock Paintings done by the people of the early stone age.'
q11 = 'Describe life in Mehrgarh'
q12 = 'Give details of Cave paintings in France.'
q13 = 'Which plants and animals were selected for domestication ?'
q14 = 'Name a famous neolithic site.'
q15 = 'Life at Catal Huyuk'
q16 = 'How were animals of help to people ?'
q17 = 'How did growing plants affect location of people ?'
q18 = 'Explain the terms Palaeolithic Age and Mesolithic age.'
q19 = 'Which animals fomred a major part of cave paintings ?'
q20 = 'Where was Tushar travelling ?'

a1 = 1
a2 = 2
a3 = 3
a4 = 5
a5 = 6
a6 = 7
a7 = 8
a8 = 9
a9 = 15
a10 = 5
a11 = 16
a12 = 17
a13 = 11
a14 = 18
a15 = 18
a16 = 13
a17 = 12
a18 = 8
a19 = 17
a20 = 1


q_list = [q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20]
a_list = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20]
for kw_met in kw_dict.values():
    correct_ans = 0
    for i in range(len(q_list)):
        for model_met in model_dict_2.values():
            qvec = Vectorize_Question(question = q_list[i], model = model_met, method = kw_met)
            p_sc = Para_Vec_Scorer(qvec, model_met, method = kw_met)
            pt_sc = Para_Title_Scorer(q_list[i])
            pid = Find_Para(p_sc, pt_sc)
            
            print(i, kw_met, pid, a_list[i])
            if pid == a_list[i]:
                correct_ans +=1
    print(correct_ans,' out of 20:  ', kw_met)
    print("----------------------------------------")

0 Spacy 1 1
1 Spacy 12 2
2 Spacy 3 3
3 Spacy 5 5
4 Spacy 6 6
5 Spacy 7 7
6 Spacy 8 8
7 Spacy 9 9
8 Spacy 3 15
9 Spacy 2 5
10 Spacy 16 16
11 Spacy 17 17
12 Spacy 11 11
13 Spacy 18 18
14 Spacy 12 18
15 Spacy 2 13
16 Spacy 2 12
17 Spacy 15 8
18 Spacy 17 17
19 Spacy 1 1
13  out of 20:   Spacy
----------------------------------------
0 Yake 1 1
1 Yake 12 2
2 Yake 4 3
3 Yake 5 5
4 Yake 6 6
5 Yake 7 7
6 Yake 8 8
7 Yake 9 9
8 Yake 2 15
9 Yake 2 5
10 Yake 15 16
11 Yake 17 17
12 Yake 11 11
13 Yake 8 18
14 Yake 15 18
15 Yake 2 13
16 Yake 2 12
17 Yake 1 8
18 Yake 17 17
19 Yake 1 1
10  out of 20:   Yake
----------------------------------------
0 RakeNLTK 1 1
1 RakeNLTK 15 2
2 RakeNLTK 8 3
3 RakeNLTK 5 5
4 RakeNLTK 6 6
5 RakeNLTK 7 7
6 RakeNLTK 8 8
7 RakeNLTK 9 9
8 RakeNLTK 2 15
9 RakeNLTK 7 5
10 RakeNLTK 16 16
11 RakeNLTK 17 17
12 RakeNLTK 13 11
13 RakeNLTK 16 18
14 RakeNLTK 12 18
15 RakeNLTK 13 13
16 RakeNLTK 2 12
17 RakeNLTK 18 8
18 RakeNLTK 5 17
19 RakeNLTK 1 1
10  out of 20:   RakeNLTK
--------

<ipython-input-14-57a7a7374ef8>:11: RuntimeWarning: invalid value encountered in true_divide
  return ans/count


1 Gensim 12 2
2 Gensim 3 3
3 Gensim 17 5
4 Gensim 1 6
5 Gensim 1 7
6 Gensim 8 8
7 Gensim 1 9
8 Gensim 4 15
9 Gensim 5 5
10 Gensim 1 16
11 Gensim 17 17
12 Gensim 1 11
13 Gensim 18 18
14 Gensim 15 18
15 Gensim 1 13
16 Gensim 2 12
17 Gensim 16 8
18 Gensim 17 17
19 Gensim 1 1
8  out of 20:   Gensim
----------------------------------------
0 Keybert-distilbert 1 1
1 Keybert-distilbert 12 2
2 Keybert-distilbert 3 3
3 Keybert-distilbert 5 5
4 Keybert-distilbert 6 6
5 Keybert-distilbert 7 7
6 Keybert-distilbert 12 8
7 Keybert-distilbert 9 9
8 Keybert-distilbert 1 15
9 Keybert-distilbert 5 5
10 Keybert-distilbert 16 16
11 Keybert-distilbert 17 17
12 Keybert-distilbert 11 11
13 Keybert-distilbert 18 18
14 Keybert-distilbert 12 18
15 Keybert-distilbert 2 13
16 Keybert-distilbert 2 12
17 Keybert-distilbert 15 8
18 Keybert-distilbert 5 17
19 Keybert-distilbert 1 1
12  out of 20:   Keybert-distilbert
----------------------------------------
0 Keybert-paraphrase 1 1
1 Keybert-paraphrase 12 2
2 Keyber

In [ ]:
## Testing The accuracy:

#kw_dict = {1: "Spacy", 2: "Yake"}
kw_dict = {1: "Spacy", 2: "Yake", 3: "RakeNLTK", 4: "Gensim", 5: "Keybert-distilbert", 6: "Keybert-paraphrase"}
sum_dict = {1: "Custom", 2: "Gensim", 3: "LexRank", 4: "Luhn", 5: "LSA"}
model_dict = {1: "model2"}
model_dict_2 = {1: model2}

q1 = "Give details of Tushar's journey"
q2 = 'Describe the life of Hunter Gatherers.'
q3 = 'How was stone wood and bones used by hunter gatherers ?'
q4 = 'Give some information about Rock paintings'
q5 = 'What are sites ?'
q6 = 'Give uses of fire'
q7 = 'What is Palaeolithic period ?'
q8 = 'What were the changes in climate and environment ?'
q9 = 'Discuss Making Stone Tools by the people of Stone Age.'
q10 = 'Discuss in short the art of Rock Paintings done by the people of the early stone age.'
q11 = 'Describe life in Mehrgarh'
q12 = 'Give details of Cave paintings in France.'
q13 = 'Which plants and animals were selected for domestication ?'
q14 = 'Name a famous neolithic site.'
q15 = 'Life at Catal Huyuk'
q16 = 'How were animals of help to people ?'
q17 = 'How did growing plants affect location of people ?'
q18 = 'Explain the terms Palaeolithic Age and Mesolithic age.'
q19 = 'Which animals fomred a major part of cave paintings ?'
q20 = 'Where was Tushar travelling ?'

a1 = 1
a2 = 2
a3 = 3
a4 = 5
a5 = 6
a6 = 7
a7 = 8
a8 = 9
a9 = 15
a10 = 5
a11 = 16
a12 = 17
a13 = 11
a14 = 18
a15 = 18
a16 = 13
a17 = 12
a18 = 8
a19 = 17
a20 = 1


q_list = [q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20]
a_list = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20]
for kw_met in kw_dict.values():
    correct_ans = 0
    for i in range(len(q_list)):
        for model_met in model_dict_2.values():
            qvec = Vectorize_Question(question = q_list[i], model = model_met, method = kw_met)
            p_sc = Para_Vec_Scorer(qvec, model_met, method = kw_met)
            pt_sc = [0]*len(p_sc)
            pid = Find_Para(p_sc, pt_sc)
            
            print(i, kw_met, pid, a_list[i])
            if pid == a_list[i]:
                correct_ans +=1
            
    print(correct_ans,' out of 20:  ', kw_met)
    print("----------------------------------------")